In [3]:
# !git clone https://github.com/ultralytics/yolov5  # clone - reference

%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 2022-8-29 Python-3.10.4 torch-1.12.1+cu102 CPU


Setup complete ✅ (8 CPUs, 15.4 GB RAM, 61.9/233.2 GB disk)


In [ ]:
!python train.py --img 640 --batch 15 --epochs 105 --data Nplate.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=Nplate.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=105, batch_size=15, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
default message here: github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 2022-8-29 Python-3.9.12 torch-1.12.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.01

      5/104         0G    0.07757      0.018    0.04534         11        640: 1
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all         25         26    0.00269      0.667     0.0178    0.00437

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      6/104         0G    0.07297    0.01782    0.04415          8        640: 1
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all         25         26    0.00369      0.927      0.118     0.0335

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      7/104         0G    0.07158    0.01787    0.04466         11        640: 1
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all         25         26    0.00387      0.927     0.0605      0.014

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      8/10

     30/104         0G     0.0415   0.008856    0.01385         16        640: 1
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all         25         26      0.756      0.798      0.877      0.456

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
     31/104         0G    0.03984   0.008943      0.013         14        640: 1
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all         25         26      0.698      0.862      0.767      0.399

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
     32/104         0G    0.04234   0.008857    0.01078         14        640: 1
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all         25         26      0.766      0.834      0.881      0.457

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
     33/10

     55/104         0G    0.02896    0.00695   0.003211         12        640: 1
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all         25         26      0.915      0.912      0.977      0.601

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
     56/104         0G    0.02996   0.007088   0.003171          9        640: 1
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all         25         26      0.904      0.981      0.988      0.561

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
     57/104         0G    0.03109   0.007213   0.002912         13        640: 1
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all         25         26      0.876      0.991      0.975      0.621

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
     58/10

     80/104         0G    0.02107   0.005646   0.001944         19        640: 1
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all         25         26       0.89      0.992      0.995      0.646

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
     81/104         0G    0.02218   0.005808   0.001968          7        640: 1
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all         25         26      0.896      0.991      0.995      0.656

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
     82/104         0G     0.0224    0.00623     0.0018         22        640: 1
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all         25         26        0.9      0.989      0.995      0.662

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
     83/10

# Detect

`detect.py` runs YOLOv5 inference on a variety of sources. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image 
                          vid.mp4  # video
                          path/  # directory
                          'path/*.jpg'  # glob
                          'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                          'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```

In [ ]:
!python detect.py --weights runs/train/exp3/weights/best.pt --img 640 --conf 0.05 --source test_1.jpeg

In [ ]:
# display.Image(filename='test_img14.jpeg', width=600)

In [ ]:
# display.Image(filename='runs/detect/exp/test_img1.jpg', width=600)

In [ ]:
# Validate
!python val.py --weights yolov5s.pt --data coco.yaml --img 640 --half

In [ ]:
!pip install easyocr